In [2]:
import sys
from pathlib import Path

import numpy as np

# Add the project root (parent of "nb") to sys.path
project_root = Path("..").resolve()
sys.path.insert(0, str(project_root))

from polypart.polytopes import get_random_polytope

In [ ]:
get_random_polytope(5, 40)

Polytope(dim=6, n_ineq=40, n_vertices=1595)

In [1]:
import subprocess
from fractions import Fraction
from functools import reduce
from math import gcd
from pathlib import Path

import numpy as np


def _lcm(a: int, b: int) -> int:
    if a == 0 or b == 0:
        return abs(a) or abs(b)
    return abs(a * b) // gcd(a, b)


def _num_den(q) -> tuple[int, int]:
    """
    Return (numerator, denominator) for q.

    Supports ints, Fractions, and gmpy2.mpq (or similar objects with
    numerator/denominator attributes).
    """
    if hasattr(q, "numerator") and hasattr(q, "denominator"):
        return int(q.numerator), int(q.denominator)
    f = Fraction(q)
    return f.numerator, f.denominator


def _row_to_integer_inequality(a_row, b_i):
    """
    Given a single inequality a_row · x <= b_i with rational entries,
    return integer coefficients [c1,...,cd,c0] representing

        c1 x1 + ... + cd xd + c0 >= 0

    in Normaliz's inhom_inequalities format.

    For A x <= b we use:
        A' x <= b'  with integer A', b'
        -A' x >= -b'
    and Normaliz row (-A', b') encodes ξ·x >= η with ξ = -A', η = -b'.
    """
    a_row = list(a_row)
    coeffs = a_row + [b_i]

    # common denominator over all coefficients and RHS
    den_lcm = reduce(_lcm, (_num_den(c)[1] for c in coeffs), 1)

    ints = []
    for c in coeffs:
        num, den = _num_den(c)
        ints.append(num * (den_lcm // den))

    a_int = ints[:-1]
    b_int = ints[-1]

    # -A' x >= -b'   ->  row (-A', b')
    return [-c for c in a_int] + [b_int]


def write_normaliz_input(A, b, path: Path) -> None:
    """
    Write a Normaliz input file describing the polyhedron P = { x in R^d : A x <= b }
    with A and b possibly rational (mpq/Fraction/int)
    so that Normaliz computes the volume.

    The file uses:
        amb_space d
        inhom_inequalities m
        <rows of (-A', b')>
        Volume
    """
    A = np.asarray(A, dtype=object)
    b = np.asarray(b, dtype=object).reshape(-1)
    m, d = A.shape
    assert b.shape[0] == m

    path = Path(path)
    with path.open("w", encoding="utf-8") as f:
        f.write(f"amb_space {d}\n")
        f.write(f"inhom_inequalities {m}\n")
        for i in range(m):
            row = _row_to_integer_inequality(A[i, :], b[i])
            # safety: must be d+1 entries
            if len(row) != d + 1:
                raise ValueError(
                    f"Row {i} has wrong length {len(row)} (expected {d + 1})"
                )
            f.write(" ".join(str(c) for c in row) + "\n")
        f.write("Volume\n")


def run_normaliz_input(in_file: Path, normaliz_exe: str = "normaliz") -> Path:
    """
    Run Normaliz on the given .in file.

    Returns the path to the corresponding .out file.
    """
    in_file = Path(in_file)
    subprocess.run([normaliz_exe, str(in_file)], check=True)
    out_file = in_file.with_suffix(".out")
    if not out_file.exists():
        raise FileNotFoundError(f"Normaliz did not create {out_file}")
    return out_file


def _extract_volume_from_out(out_file: Path) -> Fraction:
    """
    volume (Euclidean) = Vol

    This is the Z^d-normalised volume of the Ehrhart cone.
    """
    out_file = Path(out_file)
    text = out_file.read_text(encoding="utf-8")

    for line in text.splitlines():
        if line.startswith("volume (Euclidean) ="):
            parts = line.split("=")
            vol_str = parts[1].strip()
            return Fraction(vol_str)

    raise ValueError(f"No 'volume (Euclidean) =' line found in {out_file}. ")


def volume_nmz(A, b, normaliz_exe: str = "normaliz") -> Fraction:
    """
    Compute the Euclidean volume of polytope P = { x in R^d : A x <= b }
    using Normaliz via Ehrhart multiplicity.

    Returns:
        Volume as a Fraction (exact rational).
    """
    A = np.asarray(A, dtype=object)
    b = np.asarray(b, dtype=object).reshape(-1)
    in_file = Path("temp_polytope_nmz.in")
    write_normaliz_input(A, b, in_file)
    out_file = run_normaliz_input(in_file, normaliz_exe=normaliz_exe)
    return _extract_volume_from_out(out_file)


A = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]], dtype=object)
b = np.array([Fraction(1, 2), 1, 0, 0], dtype=object)

vol = volume_nmz(A, b)
print(f"Volume: {vol} = {float(vol)}")

Volume: 1/2 = 0.5


In [ ]:
from polypart.geometry import Hyperplane, Polytope


def estimate_entropy(P: Polytope, cut_hyperplane: Hyperplane):
    """
    Estimate entropy of the split using number of vertices on each side.
    """
    V = P.vertices
    a, b = cut_hyperplane.normal, cut_hyperplane.offset
    left_mask = np.dot(V, a) < b
    num_left = np.sum(left_mask)
    right_mask = np.dot(V, a) > b
    num_right = np.sum(right_mask)
    total = num_left + num_right
    p_left = num_left / total
    p_right = num_right / total
    entropy = 0.0
    if p_left > 0:
        entropy -= p_left * np.log2(p_left)
    if p_right > 0:
        entropy -= p_right * np.log2(p_right)
    return entropy


def exact_entropy(
    P: Polytope, cut_hyperplane: Hyperplane, normaliz_exe: str = "normaliz"
):
    """
    Compute exact entropy of the split using volumes on each side.
    """
    A = P.A
    b = P.b
    a, c = cut_hyperplane.normal, cut_hyperplane.offset

    # Left polytope: A x <= b, a·x <= c
    A_left = np.vstack([A, a])
    b_left = np.hstack([b, c])
    vol_left = volume_nmz(A_left, b_left, normaliz_exe=normaliz_exe)

    # Right polytope: A x <= b, -a·x <= -c
    A_right = np.vstack([A, -a])
    b_right = np.hstack([b, -c])
    vol_right = volume_nmz(A_right, b_right, normaliz_exe=normaliz_exe)

    vol_total = vol_left + vol_right
    p_left = vol_left / vol_total
    p_right = vol_right / vol_total

    entropy = 0.0
    if p_left > 0:
        entropy -= p_left * float(np.log2(p_left))
    if p_right > 0:
        entropy -= p_right * float(np.log2(p_right))
    return entropy